In [0]:
#### GOOGLE DRIVE SPECIFIC ##########################
# Make sure that you have GPU selected in the Runtime
# If you do, will print Found GPU at: /device:GPU:0
# Else go to Runtime -> Change Runtime Type

import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [0]:
# CODE SNIPPET TO ACCESS THE FILES IN GOOGLE DRIVE (GO TO BROWSER AND VERIFY)
# THEN YOU CAN ACCESS THE FILES ON LEFT SIDEBAR (copy path)
# (https://colab.research.google.com/drive/1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q#scrollTo=H4SJ-tGNkOeY)

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
# !ls "/content/drive/My Drive"


Mounted at /content/drive


In [0]:
# googlepath exists for googledrive finding the files. Can be set to empty 
# string if running on computer. Have companylist.csv in same dir. Will create
# a stock_data folder and download stocks data into that. Currently just AAPL.
googlepath = "drive/My Drive/SeniorDesign19/GANAttempts"
foldernamepath = googlepath + "/Data/32BY32"
savedpath = googlepath + "/TEDS_Presentation/savedImages/"
modelpath = googlepath + "/TEDS_Presentation/savedModels/"

In [0]:
from __future__ import print_function, division

from keras.layers import Input, Dense, Flatten, Dropout, Reshape
from keras.layers import BatchNormalization, Activation, Conv2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, load_model
from keras.optimizers import Adam

from keras.datasets import cifar10
import keras.backend as K

from keras.models import model_from_json


import matplotlib.pyplot as plt

import sys
import numpy as np
import cv2
import os
import re

import numpy as np

import random

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [0]:
def get_generator(input_layer):
  '''
  Requires the input layer as input, outputs the model and the final layer
  '''
  
  hid = Dense(128 * 16 * 16, activation='relu')(input_layer)    
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)
  hid = Reshape((16, 16, 128))(hid)

  hid = Conv2D(128, kernel_size=5, strides=1,padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)    
  #hid = Dropout(0.5)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2DTranspose(128, 4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=5, strides=1, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  #hid = Dropout(0.5)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=5, strides=1, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)
                      
  hid = Conv2D(3, kernel_size=5, strides=1, padding="same")(hid)
  out = Activation("tanh")(hid)

  model = Model(input_layer, out)
  model.summary()
  
  return model, out

In [0]:
def get_discriminator(input_layer):
  '''
  Requires the input layer as input, outputs the model and the final layer
  '''

  hid = Conv2D(128, kernel_size=3, strides=1, padding='same')(input_layer)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Flatten()(hid)
  hid = Dropout(0.4)(hid)
  out = Dense(1, activation='sigmoid')(hid)

  model = Model(input_layer, out)

  model.summary()

  return model, out

In [0]:
from keras.preprocessing import image

def generate_noise(n_samples, noise_dim):
  X = np.random.normal(0, 1, size=(n_samples, noise_dim))
  return X

def show_imgs(batchidx):
  noise = generate_noise(9, 100)
  gen_imgs = generator.predict(noise)

  fig, axs = plt.subplots(3, 3)
  count = 0
  for i in range(3):
    for j in range(3):
      # Dont scale the images back, let keras handle it
      img = image.array_to_img(gen_imgs[count], scale=True)
      axs[i,j].imshow(img)
      axs[i,j].axis('off')
      count += 1
#   plt.show()
  fig.savefig(savedpath + "{}.png".format(epoch + starting_epoch))
  plt.close()

In [0]:
# GAN creation
starting_epoch = 0

model_names = os.listdir(modelpath)
model_names.sort(key = lambda x : (int(x.split('_')[0]), x.split('_')[1][:2]))
print(model_names)

if len(model_names) != 0:
    # disc, gan, gen 
    one = model_names[-3]
    m = re.search(r'[a-z]+', one)
    assert m.group() == "disc", print("wanted: disc, got: ", m.group())
    
    two = model_names[-2]
    m = re.search(r'[a-z]+', two)
    assert m.group() == "gan", print("wanted: gan, got: ", m.group())
    
    three = model_names[-1]
    m = re.search(r'[a-z]+', three)
    assert m.group() == "gen", print("wanted: gen, got: ", m.group())
    
    img_input = Input(shape=(32,32,3))
    discriminator, disc_out = get_discriminator(img_input)
    discriminator.load_weights(modelpath + one)
    discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

    discriminator.trainable = False

    noise_input = Input(shape=(100,))
    generator, gen_out = get_generator(noise_input)
    generator.load_weights(modelpath + three)

    gan_input = Input(shape=(100,))    
    x = generator(gan_input)
    gan_out = discriminator(x)
    gan = Model(gan_input, gan_out)
    gan.summary()

    gan.load_weights(modelpath + two)
    gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')
    
    m = re.search(r'([0-9]+)?', one)
    starting_epoch = int(m.group())
else:
    img_input = Input(shape=(32,32,3))
    discriminator, disc_out = get_discriminator(img_input)
    discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

    discriminator.trainable = False

    noise_input = Input(shape=(100,))
    generator, gen_out = get_generator(noise_input)

    gan_input = Input(shape=(100,))    
    x = generator(gan_input)
    gan_out = discriminator(x)
    gan = Model(gan_input, gan_out)
    gan.summary()

    gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

['0_disc.h5', '0_gan.hdf5', '0_gen.h5', '5_disc.h5', '5_gan.hdf5', '5_gen.h5', '10_disc.h5', '10_gan.hdf5', '10_gen.h5', '15_disc.h5', '15_gan.hdf5', '15_gen.h5', '20_disc.h5', '20_gan.hdf5', '20_gen.h5', '25_disc.h5', '25_gan.hdf5', '25_gen.h5', '30_disc.h5', '30_gan.hdf5', '30_gen.h5', '35_disc.h5', '35_gan.hdf5', '35_gen.h5', '40_disc.h5', '40_gan.hdf5', '40_gen.h5', '45_disc.h5', '45_gan.hdf5', '45_gen.h5', '50_disc.h5', '50_gan.hdf5', '50_gen.h5', '55_disc.h5', '55_gan.hdf5', '55_gen.h5', '60_disc.h5', '60_gan.hdf5', '60_gen.h5', '65_disc.h5', '65_gan.hdf5', '65_gen.h5', '70_disc.h5', '70_gan.hdf5', '70_gen.h5', '75_disc.h5', '75_gan.hdf5', '75_gen.h5', '80_disc.h5', '80_gan.hdf5', '80_gen.h5', '85_disc.h5', '85_gan.hdf5', '85_gen.h5', '90_disc.h5', '90_gan.hdf5', '90_gen.h5', '95_disc.h5', '95_gan.hdf5', '95_gen.h5', '100_disc.h5', '100_gan.hdf5', '100_gen.h5', '105_disc.h5', '105_gan.hdf5', '105_gen.h5', '110_disc.h5', '110_gan.hdf5', '110_gen.h5', '115_disc.h5', '115_gan.hdf5',

In [0]:
BATCH_SIZE = 50

# # Get training images
# (X_train, y_train), (X_test, _) = cifar10.load_data()

## MODIFIDED TO TAKE IMAGES FROM FOLDER########
img_names = os.listdir(foldernamepath)
random.shuffle(img_names)
x = []
for img_name in img_names:
    if img_name is not None:
        im = np.asarray(cv2.imread(foldernamepath + "/" + img_name, cv2.IMREAD_COLOR))
        if im.shape[0] == 32 and im.shape[1] == 32:
            x.append(im)

X_train = np.asarray(x)  

# Select Cars
# X_train = X_train[y_train[:,0]==1]
print ("Training shape: {}".format(X_train.shape))

# Normalize data
X_train = (X_train - 127.5) / 127.5
 
num_batches = int(X_train.shape[0]/BATCH_SIZE)

Training shape: (4170, 32, 32, 3)


In [0]:
N_EPOCHS = 500
for epoch in range(N_EPOCHS):

  cum_d_loss = 0.
  cum_g_loss = 0.
  
  for batch_idx in range(num_batches):
    # Get the next set of real images to be used in this iteration
    images = X_train[batch_idx*BATCH_SIZE : (batch_idx+1)*BATCH_SIZE]

    noise_data = generate_noise(BATCH_SIZE, 100)
    generated_images = generator.predict(noise_data)

    # Train on soft labels (add noise to labels as well)
    noise_prop = 0.05 # Randomly flip 5% of labels
    
    # Prepare labels for real data
    true_labels = np.zeros((BATCH_SIZE, 1)) + np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
    flipped_idx = np.random.choice(np.arange(len(true_labels)), size=int(noise_prop*len(true_labels)))
    true_labels[flipped_idx] = 1 - true_labels[flipped_idx]
    
    # Train discriminator on real data
    d_loss_true = discriminator.train_on_batch(images, true_labels)

    # Prepare labels for generated data
    gene_labels = np.ones((BATCH_SIZE, 1)) - np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
    flipped_idx = np.random.choice(np.arange(len(gene_labels)), size=int(noise_prop*len(gene_labels)))
    gene_labels[flipped_idx] = 1 - gene_labels[flipped_idx]
    
    # Train discriminator on generated data
    d_loss_gene = discriminator.train_on_batch(generated_images, gene_labels)

    d_loss = 0.5 * np.add(d_loss_true, d_loss_gene)
    cum_d_loss += d_loss

    # Train generator
    noise_data = generate_noise(BATCH_SIZE, 100)
    g_loss = gan.train_on_batch(noise_data, np.zeros((BATCH_SIZE, 1)))
    cum_g_loss += g_loss

  print('  Epoch: {}, Generator Loss: {}, Discriminator Loss: {}'.format(epoch+1+starting_epoch, cum_g_loss/num_batches, cum_d_loss/num_batches))
  
  show_imgs("epoch" + str(epoch + starting_epoch))
  
  if epoch % 5 == 0:
    generator.save_weights(modelpath + "{}_gen.h5".format(epoch + starting_epoch))
    discriminator.save_weights(modelpath + "{}_disc.h5".format(epoch + starting_epoch))
    gan.save_weights(modelpath + "{}_gan.hdf5".format(epoch + starting_epoch))
  

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


  Epoch: 246, Generator Loss: 2.608170440398067, Discriminator Loss: [0.33188346 0.        ]
  Epoch: 247, Generator Loss: 2.5154886260090104, Discriminator Loss: [0.3225231 0.       ]
  Epoch: 248, Generator Loss: 2.5960344866097693, Discriminator Loss: [0.3250629 0.       ]
  Epoch: 249, Generator Loss: 2.5932011144707, Discriminator Loss: [0.3230726 0.       ]
  Epoch: 250, Generator Loss: 2.570059639861785, Discriminator Loss: [0.32142484 0.        ]
  Epoch: 251, Generator Loss: 2.682399097695408, Discriminator Loss: [0.32577348 0.        ]
  Epoch: 252, Generator Loss: 2.600976103759674, Discriminator Loss: [0.32483602 0.        ]
  Epoch: 253, Generator Loss: 2.6353662028370133, Discriminator Loss: [0.3180913 0.       ]
  Epoch: 254, Generator Loss: 2.691952577556472, Discriminator Loss: [0.32541198 0.        ]
  Epoch: 255, Generator Loss: 2.662971980600472, Discriminator Loss: [0.32146972 0.        ]
  Epoch: 256, Generator Loss: 2.653186723410365, Discriminator Loss: [0.32340